In [1]:
%pip install -U tensorflow

  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: tensorflow-io-gcs-filesystem
    Found existing installation: tensorflow-io-gcs-filesystem 0.22.0
    Uninstalling tensorflow-io-gcs-filesystem-0.22.0:
      Successfully uninstalled tensorflow-io-gcs-filesystem-0.22.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.6.0
    Uninstalling tensorboard-2.6.0:
      Successfully uninstalled tensorboard-2.6.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.7.0
    Uninstalling tensorflow-2.7.0:
      Successfully uninstalled tensorflow-2.7.0
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.55.1 requires numpy<1.22,>=1.18, but you have numpy 1.22.3 which is incompatible.
datasets 1.6.2 requires tqdm<4.50.0,>=4.27, but you have tqdm 4.62.3 which is incompatible.
You should consider upgrading via the 'C:\Users\Mustapha\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


## Imports

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
import tensorflow as tf
from tensorflow.keras import layers

print("TensorFlow version:", tf.__version__)
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

TensorFlow version: 2.7.0
1 Physical GPUs, 1 Logical GPUs


In [2]:
import pathlib
import sys


sys.path.insert(0,'..')
from src.load_dataset_fft_aug import get_splitter_dataloaders_fft

# Build model

In [10]:
model = tf.keras.models.Sequential([
    layers.Conv1D(32, kernel_size=3, activation="relu", input_shape=(201, 498)),
    layers.MaxPool1D(pool_size=4, strides=4),

    layers.Conv1D(64, kernel_size=3, activation="relu"),
    layers.MaxPool1D(pool_size=2, strides=2),

    layers.Conv1D(64, kernel_size=3, activation="relu"),
    layers.Dropout(0.5),
    
    layers.LSTM(units=40, return_sequences=True),
    
    layers.MaxPool1D(pool_size=2, strides=2),
    layers.Flatten(),
    
    layers.Dense(2)
    ])

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
# L1 loss
model.compile(optimizer=optimizer,
            loss="mae",
            metrics=["mae"]
            )

In [4]:
import numpy as np
x = tf.random.normal([1, 201, 498])

In [5]:
model.fit(x=x, y=tf.constant([[0.0, 0.0]]), epochs=1)

AttributeError: in user code:

    File "C:\Users\Mustapha\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\Mustapha\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\Mustapha\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\Mustapha\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py", line 863, in train_step
        self.optimizer.minimize(loss, self.trainable_variables, tape=tape)
    File "C:\Users\Mustapha\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\optimizer_v2\optimizer_v2.py", line 532, in minimize
        return self.apply_gra

### Model Training

Bring dataset

In [6]:
_, _, train, val = get_splitter_dataloaders_fft(noise_attenuation=0.00001)

Caching dataset: 100%|██████████| 1144/1144 [00:10<00:00, 110.00it/s]


Build a generator for the dataset in order to feed it to the model directly.

In [7]:
# y_train = []
# for i in range(len(train)):
#     y_train.append(train[i][1])

# y_val = []
# for i in range(len(val)):
#     y_val.append(val[i][1])

In [11]:
def train_generator():
    for i in range(len(train)):
        instance = train[i]
        yield instance[0], instance[1]
        
def val_generator():
    for i in range(len(val)):
        instance = val[i]
        yield instance[0], instance[1]
        
BATCH_SIZE = 32
train_dataset = tf.data.Dataset.from_generator(train_generator, output_types=(tf.float32, tf.float32), output_shapes=((201, 498), (2,))).padded_batch(BATCH_SIZE)
val_dataset = tf.data.Dataset.from_generator(val_generator, output_types=(tf.float32, tf.float32), output_shapes=((201, 498), (2,))).padded_batch(BATCH_SIZE)

Fit the model 😃

In [12]:
model.fit(train_dataset, epochs=30, validation_data=val_dataset)

Epoch 1/30


AttributeError: in user code:

    File "C:\Users\Mustapha\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\Mustapha\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\Mustapha\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\Mustapha\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py", line 863, in train_step
        self.optimizer.minimize(loss, self.trainable_variables, tape=tape)
    File "C:\Users\Mustapha\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\optimizer_v2\optimizer_v2.py", line 532, in minimize
        return self.apply_gra

In [ ]:
model.fit(val_dataset, epochs=2)

### Save model

In [ ]:
save_path = "saved/tf_model"
# tf.saved_model.save(model, save_path)

run_model = tf.function(lambda x: model(x))
BATCH_SIZE = 1
FREQS = 201
TIME_STEPS = 498
concrete_func = run_model.get_concrete_function(
    tf.TensorSpec([BATCH_SIZE, FREQS, TIME_STEPS], model.inputs[0].dtype))
model.save(save_path, save_format="tf", signatures=concrete_func)

INFO:tensorflow:Assets written to: model_not_trained/assets


INFO:tensorflow:Assets written to: model_not_trained/assets


# Tinify model

In [34]:
converter = tf.lite.TFLiteConverter.from_saved_model(save_path)

save_path_tflite = os.path.basename(save_path) + ".tflite"

converter.optimizations = [tf.lite.Optimize.DEFAULT]


tflite_model = converter.convert()
tflite_model_file = pathlib.Path(save_path_tflite)
tflite_model_file.write_bytes(tflite_model)

Estimated count of arithmetic ops: 20.144 M  ops, equivalently 10.072 M  MACs


97056